In [1]:
!pip install hyperliquid-python-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of rlp to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.1/325.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: bitarray
    Found existing installation: bitarray 1.2.2
    Uninstalling bitarray-1.2.2:
      Successfully uninstalled bitarray-1.2.2
  Attempting uninstall: pytest
    Found existing installation: pytest 6.0.1
    Uninstalling pytest-6.0.1:
      Successfully uninstalled p

In [2]:
from hyperliquid.info import Info
from hyperliquid.utils import constants

info = Info(constants.TESTNET_API_URL, skip_ws=True)
user_state = info.user_state("0xcd5051944f780a621ee62e39e493c489668acf4d")
print(user_state)

{'assetPositions': [{'position': {'coin': 'SOL', 'cumFunding': {'allTime': '-179.970787', 'sinceChange': '-179.970787', 'sinceOpen': '-179.970787'}, 'entryPx': '22.2863', 'leverage': {'type': 'cross', 'value': 20}, 'liquidationPx': None, 'marginUsed': '435.928811', 'maxLeverage': 50, 'positionValue': '8718.57623', 'returnOnEquity': '13.5333614', 'szi': '126.49', 'unrealizedPnl': '5899.582143'}, 'type': 'oneWay'}, {'position': {'coin': 'APT', 'cumFunding': {'allTime': '-38.089688', 'sinceChange': '-38.089688', 'sinceOpen': '-38.089688'}, 'entryPx': '9.8095', 'leverage': {'type': 'cross', 'value': 20}, 'liquidationPx': None, 'marginUsed': '24.38296', 'maxLeverage': 50, 'positionValue': '487.659213', 'returnOnEquity': '-1.76092821', 'szi': '54.09', 'unrealizedPnl': '-42.936642'}, 'type': 'oneWay'}, {'position': {'coin': 'ATOM', 'cumFunding': {'allTime': '-1747.25547', 'sinceChange': '0.240232', 'sinceOpen': '-1747.25547'}, 'entryPx': '10.9725', 'leverage': {'type': 'cross', 'value': 20}, 

In [ ]:
import json

import eth_account
import utils
from eth_account.signers.local import LocalAccount

from hyperliquid.exchange import Exchange
from hyperliquid.info import Info
from hyperliquid.utils import constants


def main():
    config = utils.get_config()
    account: LocalAccount = eth_account.Account.from_key(config["secret_key"])
    print("Running with account address:", account.address)
    info = Info(constants.TESTNET_API_URL, skip_ws=True)

    # Get the user state and print out position information
    user_state = info.user_state(account.address)
    positions = []
    for position in user_state["assetPositions"]:
        if float(position["position"]["szi"]) != 0:
            positions.append(position["position"])
    if len(positions) > 0:
        print("positions:")
        for position in positions:
            print(json.dumps(position, indent=2))
    else:
        print("no open positions")

    # Place an order that should rest by setting the price very low
    exchange = Exchange(account, constants.TESTNET_API_URL)
    order_result = exchange.order("ETH", True, 0.2, 1100, {"limit": {"tif": "Gtc"}})
    print(order_result)

    # Query the order status by oid
    if order_result["status"] == "ok":
        status = order_result["response"]["data"]["statuses"][0]
        if "resting" in status:
            order_status = info.query_order_by_oid(account.address, status["resting"]["oid"])
            print("Order status by oid:", order_status)

    # Cancel the order
    if order_result["status"] == "ok":
        status = order_result["response"]["data"]["statuses"][0]
        if "resting" in status:
            cancel_result = exchange.cancel("ETH", status["resting"]["oid"])
            print(cancel_result)


if __name__ == "__main__":
    main()

In [3]:
# Aevo

In [ ]:
'https://api-testnet.aevo.xyz'

In [ ]:
from random import randint
from eip712_structs import EIP712Struct, Address, Uint, Boolean, make_domain
from web3 import Web3
from eth_account import Account

# Generate Salt
salt = randint(0, 10**6)

# Limit price and amount is in 6 decimal places
decimals = 10**6
limit_price = int(100 * decimals)  # Limit price of $100
amount = int(2 * decimals)  # Size of 2 contracts

# Current timestamp in UNIX seconds
timestamp = 1690434000 

# instrument_id from /options-chain
instrument = 1

class Order(EIP712Struct):
    maker = Address()
    isBuy = Boolean()
    limitPrice = Uint(256)
    amount = Uint(256)
    salt = Uint(256)
    instrument = Uint(256)
    timestamp = Uint(256)

order_struct = Order(maker="your_address", # The wallet's main address
                    isBuy=True, # True if buy, False if sell
                    limitPrice=limit_price,
                    amount=amount,
                    salt=salt,
                    instrument=instrument,
                    timestamp=timestamp)

# Get bytes
domain = make_domain(name="Aevo Mainnet", version="1", chainId=1)
# Testnet Domain
# domain = make_domain(name='Aevo Testnet', version='1', chainId=11155111)
signable_bytes = Web3.keccak(order_struct.signable_bytes(domain=domain))

# Sign with key
key = "your_signing_key"
signature = Account._sign_hash(signable_bytes, key).signature.hex()